In [ ]:
import torch
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

train_dataset = datasets.MNIST(root='.', train=True, download=True, transform=transform)
# 依旧采用 Mini-Batch 的训练方法，batch_size=128
dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
dataloader



import torch.nn as nn

# 判别器，输入图片（像素），输出结果  0 、1
# 4 层结构，并把每层都使用全连接配上 ReLU 激活再带上 Dropout 防止过拟合。最后一层，用 Sigmoid 保证输出值是一个 0 到 1 之间的概率值
class Discriminator(nn.Module):
    # 判别器网络构建
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 1),  # 最终输出为概率值
            nn.Sigmoid()
        )

    def forward(self, x):  # 判别器的前馈函数
        out = self.model(x.reshape(x.size(0), 784))  # 数据展平传入全连接层
        out = out.reshape(out.size(0), -1)
        return out



# 生成器：输入随机噪声，生成图片

class Generator(nn.Module):
    # 生成器网络构建
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, 784),
            nn.Tanh()
        )

    def forward(self, x):
        x = x.reshape(x.size(0), 100)
        out = self.model(x)
        return out

# 如果 GPU 可用则使用 CUDA 加速，否则使用 CPU 设备计算
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")
dev


netD = Discriminator().to(dev)
netG = Generator().to(dev)
criterion = nn.BCELoss().to(dev)

lr = 0.0002  # 学习率
optimizerD = torch.optim.Adam(netD.parameters(), lr=lr)  # Adam 优化器
optimizerG = torch.optim.Adam(netG.parameters(), lr=lr)


def train_netD(netD, images, real_labels, fake_images, fake_labels):
    netD.zero_grad()
    outputs = netD(images)  # 判别器输入真实数据
    lossD_real = criterion(outputs, real_labels)  # 计算损失

    outputs = netD(fake_images)  # 判别器输入伪造数据
    lossD_fake = criterion(outputs, fake_labels)  # 计算损失

    lossD = lossD_real + lossD_fake  # 损失相加
    lossD.backward()
    optimizerD.step()
    return lossD

def train_netG(netG, netD_outputs, real_labels):
    netG.zero_grad()
    lossG = criterion(netD_outputs, real_labels)  # 判别器输出和真实数据之间的损失
    lossG.backward()
    optimizerG.step()
    return lossG


# 每一次的迭代中，首先应该训练判别器，然后训练生成器
from IPython import display
import matplotlib.pyplot as plt
from torchvision.utils import make_grid
%matplotlib inline


# 训练的目的是让判别器把生成器生成的数据当做是真实数据，label 只要 0、1 即可
# 设定一些参数方便训练代码书写
epochs = 100
for epoch in range(epochs):
    for n, (images, _) in enumerate(dataloader):  # Mini-batch 的训练方法，每次 100 个样本
        fake_labels = torch.zeros([images.size(0), 1]).to(dev)  # 伪造的数据 label 是 0
        real_labels = torch.ones([images.size(0), 1]).to(dev)  # 真实的数据 label 是 1

        noise = torch.randn(images.size(0), 100).to(dev)  # 产生生成器的输入，样本数*100 的矩阵
        fake_images = netG(noise)  # 通过生成器得到输出
        lossD = train_netD(netD, images.to(dev), real_labels,
                           fake_images, fake_labels)  # 训练判别器

        noise = torch.randn(images.size(0), 100).to(dev)  # 一组样本
        fake_images = netG(noise)  # 通过生成器得到这部分样本的输出
        outputs = netD(fake_images)  # 得到判别器对生成器的这部分数据的判定输出

        # 生成器每次都是根据噪声随机生成，然后根据 loss 进行参数调整
        lossG = train_netG(netG, outputs, real_labels)  # 训练生成器

        # 生成 64 组测试噪声样本，最终绘制 8x8 测试网格图像
        fixed_noise = torch.randn(64, 100).to(dev)
        # 为了使用 make_grid 绘图需要将数据处理成相应的形状
        fixed_images = netG(fixed_noise).reshape([64, 1, 28, 28])
        fixed_images = make_grid(fixed_images.data, nrow=8, normalize=True).cpu()
        plt.figure(figsize=(6, 6))
        plt.title("Epoch[{}/{}], Batch[{}/{}]".format(epoch+1, epochs, n+1, len(dataloader)))
        plt.imshow(fixed_images.permute(1, 2, 0).numpy())
        display.display(plt.gcf())
        display.clear_output(wait=True)